In [ ]:
import numpy as np
import pandas as pd
from scipy import misc
import matplotlib.pyplot as plt
import os
import glob

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

In [ ]:
dirlist=os.listdir("datawarehouse/")
Lev1=100
Lev11=150
Lev2=200
Lev3=300
dirlist_used=dirlist[0:Lev1]

In [ ]:
#initial input and output
Image_input=np.ones([1,500,500])
Return_output=np.ones(1)

### Example for load single stock!
#### Do not run!!!!

In [ ]:
# initial input and output
Image_input=np.ones([1,500,500])
Return_output=np.ones(1)
Iter=dirlist_used[0]
#Iter_list=glob.glob('/home/li/tmp_image_download/plotly_'+Iter+"*")
tmpReturn=pd.read_csv(Iter+".txt",header=None)
tmpReturn=np.array(tmpReturn)
for tmpi in range(1,tmpReturn.shape[0]+1):
    #read the image
    tmpImg=misc.imread('/home/li/tmp_image_download/plotly_'+Iter+"_"+str(tmpi)+".png")
    print(tmpi)
    tmpImg=rgb2gray(tmpImg)
    Image_input=np.concatenate((Image_input,tmpImg[None]),axis=0)
    Return_output=np.append(Return_output,tmpReturn[tmpi-1])

In [ ]:
print(Image_input.shape)
print(Return_output.shape)

## Series code for load bunch od data!!!

In [ ]:
#load previour data
Image_input=np.load('image_input.npy')
Return_output=np.load("return_output.npy")

In [ ]:
dirlist=os.listdir("datawarehouse/")
Lev1=100
Lev11=150
Lev2=200
Lev3=300
#dirlist_used=dirlist[0:Lev1] is finished and saved!
#dirlist_used=dirlist[Lev1:Lev11] is finished!
dirlist_used=dirlist[Lev11:Lev2]
#initial arrays
#this one has been used in lev1
#Image_input=np.ones([1,500,500])
#Return_output=np.ones(1)


for Iter in dirlist_used:
    print("starting on "+Iter)
    try:
        tmpReturn=pd.read_csv(Iter+".txt",header=None)
    except:
        print("error on "+ Iter)
        continue
    tmpReturn=np.array(tmpReturn)
    for tmpi in range(1,tmpReturn.shape[0]+1):
        #read the image
        try:
            tmpImg=misc.imread('/home/li/tmp_image_download/plotly_'+Iter+"_"+str(tmpi)+".png")
        except:
            print("error on "+ str(tmpi))
            continue
        tmpImg=rgb2gray(tmpImg)
        Image_input=np.concatenate((Image_input,tmpImg[None]),axis=0)
        Return_output=np.append(Return_output,tmpReturn[tmpi-1])

In [ ]:
Image_input=np.load('image_input.npy')
Return_output=np.load("return_output.npy")
print(Image_input.shape)
print(Return_output.shape)

In [ ]:
#save data
np.save(file="image_input",arr=Image_input)
np.save(file="return_output",arr=Return_output)

## Try for a CNN now!

In [ ]:
import numpy as np
Image_input=np.load('image_input.npy')
Return_output=np.load("return_output.npy")
print(Image_input.shape)
print(Return_output.shape)

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [ ]:
batch_size = 64
num_classes = 2
epochs = 5

# input image dimensions
img_rows, img_cols = 500, 500

#load the data
#Image_input=np.load('image_input.npy')
#Return_output=np.load("return_output.npy")

In [ ]:
from sklearn.model_selection import train_test_split
#split into training and testing set
X_train, X_test, y_train, y_test = train_test_split(Image_input, Return_output, test_size=0.1)
del Image_input,Return_output

In [ ]:
#create sign of Y
y_train_Binary=np.sign(y_train)
y_test_Binary=np.sign(y_test)
y_train_Binary[y_train_Binary==-1]=0
y_test_Binary[y_test_Binary==-1]=0

### Create a CNN model

In [ ]:
y_train_Binary = keras.utils.to_categorical(y_train_Binary,num_classes=2)
y_test_Binary = keras.utils.to_categorical(y_test_Binary,num_classes=2)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [ ]:
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
input_shape

In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(10, 10),strides=[5,5],
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(#loss=keras.losses.categorical_crossentropy,
              loss=keras.losses.categorical_hinge,
              optimizer=keras.optimizers.Adadelta(),
              #optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
batch_size = 128
num_classes = 2
epochs = 5

# input image dimensions
img_rows, img_cols = 500, 500

model.fit(X_train, y_train_Binary,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test_Binary))
score = model.evaluate(X_test, y_test_Binary, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#output predicted value
Return_pre=model.predict_classes(X_test, verbose=1)
y_test_nn=np.sign(y_test)
y_test_nn[y_test_nn==-1]=0
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test_nn, Return_pre)
confusion_matrix

## NN for continues output

In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(10, 10),strides=[5,5],
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,kernel_initializer='normal'))

model.compile(loss=keras.losses.mean_absolute_percentage_error,
              optimizer=keras.optimizers.Adadelta(),
              #optimizer='sgd',
              metrics=['accuracy'])

batch_size = 128
#num_classes = 2
epochs = 5

# input image dimensions
#img_rows, img_cols = 500, 500

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
Return_pre=model.predict_classes(X_test, verbose=1)

In [ ]:
y_test

In [ ]:
a=1
print(a)
del a

In [ ]:
a

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
import graphviz